Nomes: Bel Cogo e Bruno da Siqueira Hoffmann.

Professor: Felipe de Morais.

Disciplina: Ciência de Dados e Big Data.

# Tarefa 6

Para a tarefa em questão foi utilizado o banco NoSQL MongoDB, além disso, foi usado a base de dados: https://www.kaggle.com/datasets/evgeny1928/playstation-games-info?select=games.csv. Usando o modelo `.csv` e executando os scripts abaixo para realizar a conversão do `.csv` em `.json`. A partir disso foi feita as configurações expostas na aula para execução do banco de dados Mongo e a inserção da collection.

In [ ]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = []
        
        for row in csv_reader:
            if 'platforms' in row:
                row['platforms'] = eval(row['platforms'])
            if 'genres' in row:
                row['genres'] = eval(row['genres'])
            data.append(row)
    
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

csv_to_json('games.csv', 'games.json')